In [48]:
import pandas as pd
test = pd.read_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/분석중/데이터/원본/test.csv')
train = pd.read_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/분석중/데이터/원본/train.csv')
sub = pd.read_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/분석중/데이터/원본/sample_submission.csv')
holiday = pd.read_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/분석중/데이터/16_22공휴일.csv')
df = pd.read_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/제출용/훈영_train+test.csv')

## 인코딩

In [49]:
#공휴일 인코딩
df['공휴일'].fillna(0, inplace=True) 
df.loc[df['공휴일'].str.contains('2', na=False), '공휴일'] = 1
df.loc[df['공휴일'].str.contains('1', na=False), '공휴일'] = 2

In [50]:
#요일 인코딩
df_encoded = df.copy()
from sklearn.preprocessing import LabelEncoder
day = df_encoded['요일'].values
le = LabelEncoder()
df_encoded['요일'] = le.fit_transform(day)

In [51]:
#일자 인코딩
import pandas as pd
import numpy as np

start_date = '2016-02-01'
end_date = '2021-04-19'
date_range = pd.date_range(start_date, end_date, freq='B') # 평일 데이터만 추출
date_encoded = pd.DataFrame({'일자': date_range, '값': np.random.rand(len(date_range))})

day_diff = (date_encoded['일자'] - pd.to_datetime(start_date)).dt.days
date_encoded['코사인_일자'] = np.cos(2 * np.pi * day_diff / 365)
date_encoded['일자'] = pd.to_datetime(date_encoded['일자'])
date_encoded = date_encoded.iloc[:,[0,2]]
df_encoded['일자'] = pd.to_datetime(df_encoded['일자'])
df_encoded = pd.merge(df_encoded, date_encoded, how='left', on='일자')
df_encoded = df_encoded.iloc[:,1:]

In [53]:
df_encoded = df_encoded.iloc[:,1:]
df_encoded = df_encoded.rename(columns={'코사인_일자':'일자'})

,공휴일,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,본사근무자수,중식_밥,중식_국,중식_메인,석식_밥,석식_국,석식_메인,중식계,석식계,일자
0,0,3,2601.0,50.0,150.0,238.0,0.0,2401.0,쌀밥,오징어찌개,쇠불고기,쌀밥,육개장,자반고등어구이,1039.0,331.0,1.000000
1,0,4,2601.0,50.0,173.0,319.0,0.0,2378.0,쌀밥,김치찌개,가자미튀김,콩나물밥,어묵국,유산슬,867.0,560.0,0.999852
2,1,2,2601.0,56.0,180.0,111.0,0.0,2365.0,카레덮밥,팽이장국,치킨핑거,쌀밥,청국장찌개,황태양념구이,1017.0,573.0,0.999407
3,2,1,2601.0,104.0,220.0,355.0,0.0,2277.0,쌀밥,쇠고기무국,주꾸미볶음,미니김밥,우동,멕시칸샐러드,978.0,525.0,0.998667
4,2,0,2601.0,278.0,181.0,34.0,0.0,2142.0,쌀밥,떡국,돈육씨앗강정,쌀밥,차돌박이찌개,닭갈비,925.0,330.0,0.997630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,0,3,2973.0,125.0,174.0,704.0,331.0,2343.0,쌀밥,쇠고기미역국,춘천닭갈비,흑미밥,돈육고추장찌개,갈치구이,NaN,NaN,0.436651
1251,0,4,2973.0,76.0,170.0,636.0,364.0,2363.0,쌀밥,순두부백탕,매콤소갈비찜,추가밥,짬뽕*생면,수제찹쌀꿔바로우,NaN,NaN,0.421101
1252,0,2,2973.0,96.0,214.0,1.0,334.0,2329.0,쌀밥,냉이국,돈육간장불고기,단호박카레라이스,시금치된장국,소떡소떡,NaN,NaN,0.405426
1253,0,1,2973.0,105.0,238.0,509.0,324.0,2306.0,쌀밥,맑은떡국,가자미구이,흑미밥,어묵매운탕,쇠고기숙주볶음,NaN,NaN,0.389630


## 모델학습

In [54]:
df_encoded = df_encoded[['일자', '공휴일', '요일', '본사정원수', '본사휴가자수', '본사출장자수',
       '본사시간외근무명령서승인건수', '본사근무자수', '중식계', '석식계']]

In [55]:
label = ['중식계', '석식계']
df_encoded['공휴일'] = df_encoded['공휴일'].astype(int)
df_encoded[''] = df_encoded['공휴일'].astype(int)
y = df_encoded[label]
X = df_encoded.drop(columns=label)
X['공휴일'] = X['공휴일'].astype(int)

In [56]:
X_train = X.iloc[:1205]
X_test = X.iloc[1205:]
y_train = y.iloc[:1205]
y_test = y.iloc[1205:]

In [57]:
lunch_X_train = X_train[['일자', '공휴일', '요일', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
        '본사근무자수']]
lunch_X_test = X_test[['일자', '공휴일', '요일', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
        '본사근무자수']]
lunch_y_train = y_train['중식계']
lunch_y_test = y_test['중식계']
dinner_set = df_encoded.iloc[:1205]
dinner_set = dinner_set[dinner_set['석식계']!=0]
dinner_X_train = dinner_set[['일자', '공휴일', '요일', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
        '본사근무자수']]
dinner_y_train = dinner_set['석식계']
dinner_X_test = X_test[['일자', '공휴일', '요일', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '본사근무자수']]
dinner_y_test = y_test['석식계']

In [58]:
import numpy as np
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

xgb_clf = xgb.XGBClassifier()

In [59]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100),
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}
model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(lunch_X_train, lunch_y_train)
print(grid_search.best_params_)
{'colsample_bylevel': 0.5, 'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 600}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

{'colsample_bylevel': 0.5, 'colsample_bytree': 0.5, 'max_depth': 2, 'n_estimators': 300}


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.5, colsample_bynode=None, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=0,
             max_depth=3, max_leaves=None, min_child_weight=1, missing=None,
             monotone_constraints=None, n_estimators=600, n_jobs=1,
             nthread=None, num_parallel_tree=None, objective='reg:linear', ...)

In [60]:
lunch_y_pred = grid_search.predict(lunch_X_test)
sub['중식계'] = lunch_y_pred

In [61]:
%%time 

from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

d_model = xgb.XGBRegressor()
d_grid_search = GridSearchCV(estimator=d_model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

d_grid_search.fit(dinner_X_train, dinner_y_train)
print(d_grid_search.best_params_)
{'colsample_bylevel': 0.5, 'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 600}

{'colsample_bylevel': 0.5, 'colsample_bytree': 0.7, 'max_depth': 2, 'n_estimators': 300}
CPU times: user 3.59 s, sys: 189 ms, total: 3.78 s
Wall time: 2min 50s


{'colsample_bylevel': 0.5,
 'colsample_bytree': 0.5,
 'max_depth': 3,
 'n_estimators': 600}

In [62]:
dinner_y_pred = d_grid_search.predict(dinner_X_test)
sub['석식계'] = dinner_y_pred
sub

,일자,중식계,석식계
0,2021-01-27,1031.615356,436.528778
1,2021-01-28,1036.311768,510.821014
2,2021-01-29,733.638855,241.422760
3,2021-02-01,1327.508545,548.994019
4,2021-02-02,1043.202271,566.994507
5,2021-02-03,1037.940674,397.550446
6,2021-02-04,958.902283,477.693787
7,2021-02-05,672.599121,366.821869
8,2021-02-08,1299.107056,663.519836
9,2021-02-09,1084.767578,552.302612


In [63]:
sub.to_csv('/content/drive/Shareddrives/네번째_으싸으싸팀/제출용/훈영_성능개선1차-xgb.csv', index=False)